Broadcast variables in Databricks (Apache Spark) are used to efficiently distribute read-only variables to worker nodes, reducing communication costs during task execution. Understanding when and how to use broadcast variables can significantly impact performance, especially when dealing with large datasets or frequent joins.

Broadcast Variable Usage

Without Broadcast Variable

When you perform operations without using broadcast variables, Spark may shuffle data across the network multiple times, which can lead to increased network overhead and slower performance. For instance, consider the following example without using a broadcast variable:

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Broadcast Variable Example").getOrCreate()

# Sample large DataFrame
large_df = spark.range(100)

# Broadcast variable example
broadcast_var = spark.sparkContext.broadcast([1, 2, 3, 4, 5])

# Function to filter data using broadcast variable
def filter_data(value):
    return value in broadcast_var.value

# Apply filter operation without broadcast variable
filtered_data = large_df.filter((large_df["id"]>1) & (large_df["id"]<4)).display()


id
2
3


With Broadcast Variable:

Using a broadcast variable optimizes performance by distributing the variable to worker nodes only once, thereby reducing network traffic and improving execution speed. Here’s how you can modify the previous example to use a broadcast variable:

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType

# Initialize Spark session
spark = SparkSession.builder.appName("Broadcast Variable Example").getOrCreate()

# Sample large DataFrame
large_df = spark.range(100)

# Broadcast variable example
broadcast_var = spark.sparkContext.broadcast([1, 2, 3, 4, 5])

# Function to filter data using broadcast variable
def filter_data(value):
    return value in broadcast_var.value

# Register UDF
filter_data_udf = udf(filter_data, BooleanType())

# Apply filter operation with broadcast variable
filtered_data = large_df.filter(filter_data_udf(col("id")))
filtered_data.display()


id
1
2
3
4
5


Performance Impact
Without Broadcast Variable: Spark may shuffle data across the network multiple times, especially during operations like joins or filters involving large datasets, leading to higher network overhead and slower performance.

With Broadcast Variable: The broadcast variable is sent to each executor once and reused across multiple tasks, reducing the amount of data shuffled over the network and improving overall performance.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

# Initialize Spark session
spark = SparkSession.builder.appName("Broadcast Join Example").getOrCreate()

# Sample small DataFrame
small_df = spark.createDataFrame([(1, "A"), (2, "B"), (3, "C")], ["id", "value"])

# Sample large DataFrame
large_df = spark.range(1000).toDF("id")

# Perform broadcast join
joined_df = large_df.join(broadcast(small_df), "id")

# Show the results
joined_df.display()


id,value
1,A
2,B
3,C


###Guidelines for Using Broadcast Variables

Size Consideration: Use broadcast variables for read-only data that is small enough to fit in memory across all worker nodes. This typically includes lookup tables or small datasets used in joins or filters.

Optimal Use Cases: Broadcast variables are particularly effective when used with operations like joins (broadcastHashJoin) or when filtering large DataFrames based on a small set of values.

Broadcast Joins: When performing joins, consider broadcasting smaller DataFrames to optimize performance, especially when joining large and small DataFrames.

Conclusion:

Broadcast variables in Databricks (Apache Spark) are a powerful optimization technique for reducing network overhead and improving performance, especially when dealing with large datasets and frequent data accesses. By leveraging broadcast variables judiciously, you can significantly enhance the efficiency of your Spark jobs on Databricks.